In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow_datasets
import tensorflow

In [4]:
dataset, info = tensorflow_datasets.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteFAZF7Q/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteFAZF7Q/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteFAZF7Q/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [10]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [11]:
encoder = info.features['text'].encoder

In [12]:
BUFFERSIZE = 10000
BATCHSIZE = 64
padded_shapes = ([None], ())
train_dataset = train_dataset.shuffle(BUFFERSIZE).padded_batch(BATCHSIZE, padded_shapes = padded_shapes)
test_dataset = test_dataset.padded_batch(BATCHSIZE, padded_shapes = padded_shapes)

In [28]:
model = tensorflow.keras.Sequential([tensorflow.keras.layers.Embedding(encoder.vocab_size, 64),
                                    tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(64, return_sequences=True)),
                                    tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(32)),
                                    tensorflow.keras.layers.Dense(64, activation='relu'),
                                    tensorflow.keras.layers.Dropout(0.5),
                                    tensorflow.keras.layers.Dense(1, activation='sigmoid')]
                                    )

In [29]:
model.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(1e-4), metrics=['accuracy'])

In [30]:
history = model.fit(train_dataset, epochs=5, validation_data=test_dataset, validation_steps=30)

Epoch 1/5
391/391 [==============================] - 78s 187ms/step - loss: 0.6539 - accuracy: 0.5830 - val_loss: 0.4875 - val_accuracy: 0.7776
Epoch 2/5
391/391 [==============================] - 73s 185ms/step - loss: 0.4233 - accuracy: 0.8258 - val_loss: 0.3885 - val_accuracy: 0.8396
Epoch 3/5
391/391 [==============================] - 74s 188ms/step - loss: 0.3133 - accuracy: 0.8856 - val_loss: 0.3516 - val_accuracy: 0.8609
Epoch 4/5
391/391 [==============================] - 74s 187ms/step - loss: 0.2593 - accuracy: 0.9110 - val_loss: 0.3488 - val_accuracy: 0.8583
Epoch 5/5
391/391 [==============================] - 73s 186ms/step - loss: 0.2300 - accuracy: 0.9239 - val_loss: 0.3692 - val_accuracy: 0.8672


In [31]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

In [32]:
def sample_predict(sentence, pad=True):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tensorflow.cast(encoded_sample_pred_text, tensorflow.float32)
  prediction = model.predict(tensorflow.expand_dims(encoded_sample_pred_text, 0))

  return prediction

In [37]:
sample_text = "Movie was bad. Will not recommend it to anyone. Bad script and visuals"
prediction = sample_predict(sample_text, pad=True) * 100
print("Probability of Positive Review : %.2f" % prediction)

Probability of Positive Review : 11.92


In [38]:
sample_text = "Great movie. Fun to watch. Definitely recommended"
prediction = sample_predict(sample_text, pad=True) * 100
print("Probability of Positive Review : %.2f" % prediction)

Probability of Positive Review : 85.31
